In [ ]:
import os, pickle
import numpy as np
import tensorflow as tf
from keras.callbacks import *
#from keras.optimizers.optimizer_experimental.adamw import AdamW

from settings import *
import data
from generator import *
from model import *
from callbacks import *
from utils import *
from tokenizer import *
import pretokenizer as mypretk

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:

	try:
		tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
		print('Using GPU :)')

	except RuntimeError as e:
		print(e)

else:
	print('Using CPU :(')

In [ ]:
dataset = data.parse_dataset(DATASET_PATH)

print('Dataset size:', '{:,.0f}'.format(len(dataset)))

print('\n' + dataset[:500])

In [ ]:
words = mypretk.split(dataset[:500])

print(dataset[:500].replace(' ', '_'))
print_tokens(words)

In [ ]:
tokenizer = Tokenizer(dataset)
vocab = tokenizer.vocab

print('\nVocab size:', '{:,.0f}'.format(len(vocab)), '\n')

for v in vocab:
	print(f'[{v}]', end = ' ')

In [ ]:
if os.path.exists(os.path.join(PROCESSED_DATA_DIR, 'tokens.npy')):
	tokens = np.load(os.path.join(PROCESSED_DATA_DIR, 'tokens.npy'))

else:
	tokens = np.array(tokenizer.encode(dataset).ids)

	if not os.path.exists(PROCESSED_DATA_DIR):
		os.mkdir(PROCESSED_DATA_DIR)

	np.save(os.path.join(PROCESSED_DATA_DIR, "tokens.npy"), tokens)

print('Encoded dataset:')

for token in tokens[:100]:
	print(token, end = " ")

print('...')
print('\n\nDecoded dataset:')
print('"', end = '')

for i in tokens[:100]:
	text = tokenizer.decode([i], False)
	if text == '<eom>':
		print('\n')
	elif text == '<eod>':
		print('\n-------------------\n')
	else:
		print(text, end = '')

print('..."')

In [ ]:
train_indexes, val_indexes = data.split_dataset(tokens)

print("Train indexes:", '{:,.0f}'.format(len(train_indexes)))
print("Val indexes:", '{:,.0f}'.format(len(val_indexes)))

In [ ]:
train_generator = BatchGenerator(tokens, train_indexes, val_frequency = VAL_FREQUENCY)
x, y = train_generator.__getitem__(0)

for i in range(len(x[0])):
	print(x[0][i], end = (max(len(str(x[0][i])), len(str(y[0][i]))) - len(str(x[0][i])) + 1) * " " + "| ")

print()

for i in range(len(y[0])):
	print(y[0][i], end = (max(len(str(x[0][i])), len(str(y[0][i]))) - len(str(y[0][i])) + 1) * " " + "| ")

print("\n")
print('"' + decode(tokenizer, x[0]) + '"')
print('"' + decode(tokenizer, y[0]) + '"')

In [ ]:
model = create_model(len(vocab))

model.compile(
	optimizer = AdamW(learning_rate = 0, weight_decay = WEIGHT_DECAY, beta_1 = BETA_1, beta_2 = BETA_2, global_clipnorm = CLIP_GRADIENTS),
	loss = 'sparse_categorical_crossentropy',
	metrics = ['accuracy']
)

model.summary()

In [ ]:
model.fit(
	train_generator,
	validation_data = BatchGenerator(tokens, val_indexes, ratio = 1.0 / VAL_FREQUENCY),
	batch_size = BATCH_SIZE,
	validation_batch_size = BATCH_SIZE,
	epochs = NUM_EPOCHS,
	shuffle = False,
	callbacks = [
		LRScheduler(len(train_indexes) // VAL_FREQUENCY),
		ModelCheckpoint(
			filepath = "model.h5",
			monitor = "val_loss",
			save_best_only = True,
			save_weights_only = True
		),
		EarlyStopping(
			monitor = "val_loss",
			patience = 20,
			restore_best_weights = True
		)
	]
)

In [ ]:
model.load_weights("model.h5")

In [ ]:
model.evaluate(BatchGenerator(tokens, val_indexes, ratio = 1.0 / VAL_FREQUENCY), batch_size = BATCH_SIZE)

In [ ]:
input = "<eod>"
output = predict(model, input, tokenizer, 256, 1, True)